In [1]:
import sys
sys.path.insert(1, '/home/b3arjuden/crocket')

In [2]:
from json import load as json_load
from random import randint, shuffle
from requests import get
from time import sleep, time
from requests import get
from requests_futures.sessions import FuturesSession
from requests.exceptions import ConnectTimeout, ProxyError, ReadTimeout
from concurrent.futures import as_completed, Future

from crocket.bittrex.bittrex2 import Bittrex, return_request_input

In [3]:
PROXY_LIST_PATH = '/home/b3arjuden/crocket/proxy_list.txt'
BITTREX_CREDENTIALS_PATH = '/home/b3arjuden/bittrex_credentials.json'

In [4]:
def configure_ip(ip):
    
    return {
        'http': ip,
        'https': ip
    }

def process_response(session, response):
    
    try:
        response.data = response.json()
    except:
        response.data = {
               'success': False,
               'message': 'NO_API_RESPONSE',
               'result': None
            }

In [5]:
# Read files

with open(PROXY_LIST_PATH, 'r') as f:
    proxies = f.read().splitlines()
    
with open(BITTREX_CREDENTIALS_PATH, 'r') as f:
    credentials = json_load(f)

In [6]:
# Create bittrex objects

b1 = Bittrex(api_key=credentials.get('key'), 
             api_secret=credentials.get('secret'), 
             api_version='v1.1')

b2 = Bittrex(api_key=credentials.get('key'), 
             api_secret=credentials.get('secret'), 
             dispatch=return_request_input,
             api_version='v1.1')

In [7]:
# Get list of currencies

response = b1.get_markets()

currencies = [x.get('MarketName') for x in response.get('result') 
              if x.get('BaseCurrency') == 'BTC' and x.get('IsActive')]

markets = currencies

In [8]:
rind = list(range(200))

In [ ]:
# Test asynchronous requests

with FuturesSession(max_workers=20) as session:

    for ii in range(0, 10): # TODO: replace with while loop

        shuffle(rind)

        futures = []

        start = time()

        for index in range(len(markets)):

            market = markets[index]
            request_input = b2.get_market_history(market)

            proxy = configure_ip(proxies[rind[index]])
            url = request_input.get('url')
            headers = {"apisign": request_input.get('apisign')}
            
            response = session.get(url, 
                                   background_callback=process_response,
                                   headers=headers,
                                   timeout=5,
                                   proxies=proxy)
            
            # Add attributes to response
            response.market = market
            response.url = request_input.get('url')
            response.headers = headers
            response.proxy = proxy
            
            futures.append(response)

        stop = time()
        print('Total elapsed time: {}s'.format(str(stop-start)))

        start = time()
        for future in as_completed(futures):
            
            try:
                data = future.result().data
                
            except (ProxyError, ConnectTimeout, ReadTimeout):
                print('Failed API call for {}. Proxy: {}.'.format(future.market, future.proxy.get('https')))
                
                futures.remove(future)
                
                while True:
                    try:
                        response = session.get(future.url, 
                                               background_callback=process_response,
                                               headers=future.headers,
                                               timeout=5,
                                               proxies=future.proxy)
                        sleep(3)
                        data = response.result().data
                        futures.append(response)
                    except (ProxyError, ConnectTimeout, ReadTimeout):
                        print('Retried API call failed.')
                        pass
                
                print('Retried API call for {} successful.'.format(future.market))
        
        stop = time()
        print('Execution time for all futures: {}s'.format(str(stop-start)))

        sleep(30)

Total elapsed time: 0.05339837074279785s
Failed API call for BTC-VIB. Proxy: 173.208.103.175:8800.
Retried API call failed.
Retried API call failed.


In [ ]:
tmp = [1, 2, 3, 4, 5]

for x in tmp:
    print(x)
    
    x = 3
    
print(tmp)

In [ ]:
futures.remove(futures[1])

In [ ]:
futures